In [1]:
from analysis.pre_process import DataProcess

# 默认读取内置的数据集
process = DataProcess()

In [2]:
debt_df = process.read_data('debt')

长期借款(元), 应付债券(元), 以公允价值计量且其变动计入当期损益的金融资产(元), 债权投资(元), 持有至到期投资(元), 其他非流动金融资产(元), 以上字段未找到，已自动赋值为0。

！以下字段须手动赋值：其他流动资产里的理财产品,其他流动资产里的结构性存款,长期应付款,应收款项融资,合同负债(元),合同资产(元)


In [3]:
# 18年及之前的科目
debt_df['其他流动资产里的理财产品'] = [0, 0, 1500000000, 2570000000, 0, 0]
debt_df['其他流动资产里的结构性存款'] = [0, 0, 0, 0, 0, 0]

# 用来计算有息负债，如果是无息则直接写 0 即可
debt_df['长期应付款'] = [0, 0, 0, 0, 0, 0]

# 2019 年及之后的科目
debt_df['应收款项融资'] = [0, 0, 0, 0, 408972104.07, 0]
debt_df['合同资产(元)'] = [0, 0, 0, 0, 0, 0]
debt_df['合同负债(元)'] = [0, 0, 0, 0, 0, 949591228.35]

In [4]:
benefit_df = process.read_data('benefit')
cash_df = process.read_data('cash')

### 2017 年及之前利息收入和利息费用尚未分开披露

财务费用如果是正数，就将其归到利息费用中；相反如果是负数，就是利息收入。

In [5]:
index_arr = benefit_df[benefit_df['利息收入(元)'] == 0].index
index_arr

Int64Index([2015, 2016], dtype='int64')

In [6]:
for index in index_arr:
    if benefit_df.loc[index, '财务费用(元)'] < 0:
        benefit_df.loc[index, '利息收入(元)'] = benefit_df.loc[index, '财务费用(元)'] * -1
benefit_df['利息收入(元)']

2015    65957900.00
2016    78635300.00
2017    93430000.00
2018   100909000.00
2019    84590700.00
2020   160282600.00
Name: 利息收入(元), dtype: float64

In [7]:
index_arr = benefit_df[benefit_df['其中：利息费用(元)'] == 0].index
index_arr

Int64Index([2015, 2016, 2017, 2018], dtype='int64')

In [8]:
for index in index_arr:
    if benefit_df.loc[index, '财务费用(元)'] > 0:
        benefit_df.loc[index, '其中：利息费用(元)'] = benefit_df.loc[index, '财务费用(元)']
benefit_df['其中：利息费用(元)']

2015         0.00
2016         0.00
2017         0.00
2018         0.00
2019    477400.00
2020   6721500.00
Name: 其中：利息费用(元), dtype: float64

In [9]:
process.save_data()